- 만든 데이터 목록
    - song_meta_data_v3.json
    - playlist_data_v3.json

- 총 노래 수
    - 169114개

- 제거된 노래
    - ['GN1600', 'GN1800', 'GN1900', 'GN2000', 'GN2100', 'GN2200', 'GN2300', 'GN2400', 'GN2600', 'GN2700', 'GN2800', 'GN2900',] 에 포함되는 장르
    - artist_name_basket이 Various Artists 인 경우
    - 노래 제목에 Instrumental, inst., (MR), ASMR 가 포함된 경우
    - playlist에 포함되지 않은 노래
    - playlist에 포함되는 노래의 빈도 수가 2개 이하인 노래
    - 중복된 노래 (중복된 노래도 단일의 노래로 플레이리스트 교체함)

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
import gc
import warnings

warnings.filterwarnings(action='ignore')

data_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Data/'

# 데이터 불러오기

In [ ]:
playlist_df = pd.read_json(data_dir + 'train.json', typ = 'frame')
genre_df = pd.read_json(data_dir + 'genre_gn_all.json', typ = 'series')
genre_df = pd.DataFrame(genre_df, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})
song_df = pd.read_json(data_dir + 'song_meta.json', typ = 'frame')
song_df['id'] = song_df['id'].astype(str)
crawling_song_df = pd.read_json(data_dir + 'melon_crawling_data.json')

# 전처리 함수 정의

In [ ]:
def get_cnt(play_df):
    cnt = Counter()
    playlist_li = []
    play_li = play_df['songs'].values

    for play in tqdm(play_li):
        playlist_li.append(list(map(str, play)))

    for play in tqdm(playlist_li):
        cnt.update(play)
    
    return cnt

# 플레이리스트에 c 이상만 존재하는 곡만 남김 존재하는 곡 제거
def get_cnt_clean(df, play_df, c = 2):
    song_df = df.copy()
    cnt = get_cnt(play_df = play_df)
    not_del_song_id_li = []

    for song_id, song_cnt in cnt.items():
        if song_cnt >= c :
            not_del_song_id_li.append(song_id)

    print('남은 곡:', len(not_del_song_id_li))
    return song_df.set_index('id').loc[not_del_song_id_li, :].reset_index()

# 데이터 합치기
def get_merge(df, crawling_df):
    song_df = df.copy()
    crawling_song_df = crawling_df.copy()
    song_df['like'] = crawling_song_df['좋아요'].apply(lambda x : int(str(x).replace(',', '')) if not pd.isna(x) else 0).values
    return song_df.reset_index(drop = True)

# 해당 컬럼의 아티스트 및 장르 제거 함수
def get_clean_df(df, del_col, del_values):
    '''
    아티스트의 경우 
    del_col = 'artist_name_basket'
    del_values = ['Various Artists']

    장르의 경우
    del_col = 'song_gn_gnr_basket'
    del_values = ['GN2800', 'GN2700', 'GN2400', 'GN2300', 'GN2200', 'GN9000']
    '''
    del_song_idx = []
    song_df = df.copy()
    val_li = df[del_col].tolist()
    for idx, val in enumerate(val_li):
        for v in val:
            if v in del_values:
                del_song_idx.append(idx)
                break

    song_idx = list(set(song_df.index.tolist()) - set(del_song_idx))
    song_df = song_df.iloc[song_idx, :]

    return song_df.reset_index(drop = True)

def get_clean_gnr(df, del_col, del_values):
    del_song_idx = []
    song_df = df.copy()
    val_li = df[del_col].tolist()
    for idx, val in enumerate(val_li):
        temp_cnt = len(val)
        cnt = 0
        for v in val:
            if v in del_values:
                cnt += 1
        if temp_cnt == cnt:
            del_song_idx.append(idx)

    song_idx = list(set(song_df.index.tolist()) - set(del_song_idx))
    song_df = song_df.iloc[song_idx, :]

    return song_df.reset_index(drop = True)

# 플레이리스트에 존재하지 않는 노래 제거
def get_clean_playlist(df, playlist_df):
    song_df = df.copy()

    cnt = get_cnt(play_df = playlist_df)
    
    all_song = set(song_df['id'].tolist())
    playlist_in_song = set(cnt.keys())
    playlist_not_in_song = all_song - playlist_in_song

    print('전체 노래의 수:', len(all_song))
    print('플레이리스트 O 노래의 수:', len(playlist_in_song))
    print('플레이리스트 X 노래의 수:', len(playlist_not_in_song))

    return song_df.set_index('id').loc[list(playlist_in_song), :].reset_index()

# 장르 맵핑 함수
def get_kor_gnr(df, genre_df):
    song_df = df.copy()

    gnr_code2gnr_name = {}
    for gnr_code, gnr_name in zip(genre_df['gnr_code'].values, genre_df['gnr_name'].values):
        gnr_code2gnr_name[gnr_code] = gnr_name
    gnr_code2gnr_name['GN9000'] = 'etc'

    def get_kor_song_gn_gnr_basket(x):
        ret = []
        for gnr_code in x:
            try:
                ret.append(gnr_code2gnr_name[gnr_code])
            except:
                print(x)
        
        return ret

    song_df['kor_song_gn_gnr_basket'] = song_df['song_gn_gnr_basket'].apply(lambda x : get_kor_song_gn_gnr_basket(x))

    return song_df.reset_index(drop = True)

# 해당 단어 제거
def get_clean_str(df):
    song_df = df.copy()
    inst1_idx = song_df[song_df['song_name'].apply(lambda x : True if 'Instrumental'.lower() in x.lower() else False)].index.tolist()
    inst2_idx = song_df[song_df['song_name'].apply(lambda x : True if 'inst.'.lower() in x.lower() else False)].index.tolist()
    inst3_idx = song_df[song_df['song_name'].apply(lambda x : True if '(MR)' in x else False)].index.tolist()
    inst4_idx = song_df[song_df['song_name'].apply(lambda x : True if 'ASMR' in x else False)].index.tolist()

    idx_li = list(set(inst1_idx) | set(inst2_idx) | set(inst3_idx) | set(inst4_idx))
    idx_li = list(set(song_df.index.tolist()) - set(idx_li))

    return song_df.iloc[idx_li, :].reset_index(drop = True)

# 중복 노래 제거 및 플레이리스트 곡 교체
def get_clean_song_and_re_playlist(df, play_df):
    cols = df.columns.tolist()
    befor_song_df = df.copy()
    befor_playlist_df = play_df.copy()
    after_playlist_df = play_df.copy()

    befor_song_df['concat_artist_name_basket'] = befor_song_df['song_name'] + '|' + befor_song_df['artist_name_basket'].apply(lambda x : '|'.join(x))
    befor_song_df = befor_song_df.sort_values('like', ascending = False)

    song_id_li = []
    only_one_song_id_li = []
    group_df = befor_song_df.groupby(['concat_artist_name_basket'])
    for k, gp in tqdm(group_df):
        id_li = gp['id'].tolist()
        song_id_li += id_li
        only_one_song_id_li += [id_li[0]] * len(id_li)
    
    song_id2only_one_song_id = {}
    for song_id, only_one_song_id in zip(song_id_li, only_one_song_id_li):
        song_id2only_one_song_id[song_id] = only_one_song_id
    
    after_song_df = befor_song_df.set_index('id').loc[list(set(song_id2only_one_song_id.values())), :].reset_index()
    after_song_df = after_song_df[cols]

    befor_song_li = set(befor_song_df['id'].tolist())
    after_song_li = set(after_song_df['id'].tolist())

    def get_clean_song(song_li):

        song_li = set(map(str, song_li))
        song_li = list(befor_song_li & song_li)
        song_li = set(map(lambda x : song_id2only_one_song_id[x], song_li))
        song_li = list(after_song_li & song_li)

        return song_li

    after_playlist_df['songs'] = after_playlist_df['songs'].apply(get_clean_song)
    after_playlist_df = after_playlist_df[after_playlist_df['songs'].apply(lambda x : len(x)) >= 1]
    after_playlist_df = after_playlist_df.reset_index(drop = True)
    
    cnt1 = get_cnt(play_df = befor_playlist_df)
    cnt2 = get_cnt(play_df = after_playlist_df)

    for cnt1_set, cnt2_set in zip(cnt1.most_common(n = 30), cnt2.most_common(n = 30)):
        cnt1_song_id, cnt1_count = cnt1_set
        cnt2_song_id, cnt2_count = cnt2_set
        
        cnt1_df = befor_song_df[befor_song_df['id'] == cnt1_song_id]
        cnt1_song_name = cnt1_df['song_name'].values[0]
        cnt1_artist_name_basket = cnt1_df['artist_name_basket'].values[0]
        cnt1_like = cnt1_df['like'].values[0]

        cnt2_df = befor_song_df[befor_song_df['id'] == cnt2_song_id]
        cnt2_song_name = cnt2_df['song_name'].values[0]
        cnt2_artist_name_basket = cnt2_df['artist_name_basket'].values[0]
        cnt2_like = cnt2_df['like'].values[0]
        
        print(f'befor / song_id : {cnt1_song_id}, count : {cnt1_count}, song_name : {cnt1_song_name}, artist_name_basket : {cnt1_artist_name_basket}, like : {cnt1_like}')
        print(f'after / song_id : {cnt2_song_id}, count : {cnt2_count}, song_name : {cnt2_song_name}, artist_name_basket : {cnt2_artist_name_basket}, like : {cnt2_like}')
        print()
    
    return after_song_df, after_playlist_df

# 데이터 전처리

In [ ]:
# 데이터 합치기
song_df = get_merge(df = song_df, crawling_df = crawling_song_df)

In [ ]:
# 메모리 정리
del crawling_song_df

gc.collect()

In [ ]:
# 플레이리스트에 존재하지 않는 노래 제거
song_df = get_clean_playlist(df = song_df, playlist_df = playlist_df)

In [ ]:
# 중복 노래 제거 및 플레이리스트 곡 교체
song_df, playlist_df = get_clean_song_and_re_playlist(df = song_df, play_df = playlist_df)

In [ ]:
# 해당 아티스트 제거
song_df = get_clean_df(df = song_df, del_col = 'artist_name_basket', del_values = ['Various Artists'])

# 해당 장르 제거
gnr_li = ['GN1600', 'GN1800', 'GN1900', 'GN2000', 'GN2100', 'GN2200', 'GN2300', 'GN2400', 'GN2600', 'GN2700', 'GN2800', 'GN2900',]
song_df = get_clean_gnr(df = song_df, del_col = 'song_gn_gnr_basket', del_values = gnr_li)

# 해당 단어 제거
song_df = get_clean_str(df = song_df)

In [ ]:
# 중복 노래 제거 및 플레이리스트 곡 교체
song_df, playlist_df = get_clean_song_and_re_playlist(df = song_df, play_df = playlist_df)

In [ ]:
# 플레이리스트에 존재하지 않는 노래 제거
song_df = get_clean_playlist(df = song_df, playlist_df = playlist_df)

In [ ]:
# 플레이리스트에 c 이상만 존재하는 곡만 남김 존재하는 곡 제거
song_df = get_cnt_clean(df = song_df, play_df = playlist_df, c = 3)

In [ ]:
# 중복 노래 제거 및 플레이리스트 곡 교체
song_df, playlist_df = get_clean_song_and_re_playlist(df = song_df, play_df = playlist_df)

In [ ]:
# 플레이리스트에 존재하지 않는 노래 제거
song_df = get_clean_playlist(df = song_df, playlist_df = playlist_df)

In [ ]:
# 장르 맵핑 함수
song_df = get_kor_gnr(df = song_df, genre_df = genre_df)

In [ ]:
# 데이터 저장
song_df.to_json(data_dir + 'song_meta_data_v1.json')
playlist_df.to_json(data_dir + 'playlist_data_v1.json')